# SIR_Model_Simulations

Second Dashboard
* goal of the SIR model simulation is to model the susceptible individuals to get infected through contact with eachother

In [13]:
from scipy import optimize # we will use curve_fit to find the best fit to data points
from scipy import integrate #Integrate a system of ordinary differential equations
import subprocess
import os
import pandas as pd
import numpy as np
import dash
import dash_core_components as dcc # very useful for dash user interactions
import dash_html_components as html
from dash.dependencies import Input, Output,State
import plotly.graph_objects as go

# 1. SIR_Model_Functions

In [14]:
# importing the full datset from processed data folder
df_input_large = pd.read_csv(r'..\data\processed/COVID_final_set.csv',sep=';')

# sort the dataset
df_input_large.sort_values('date',ascending=True).head() 

#initializing the country list
country_list = 'Germany'
df_plot=df_input_large[df_input_large['country']==country_list]

# ignoring the first x days value
ydata = np.array(df_plot.confirmed[25: ])

# initializing the fixed parameters
N0=1000000 # max susceptible population
beta=0.4   # infection spread dynamics
gamma=0.1  # recovery rate

# parameters used in the dynamic beta simulations (hard assumptions), i will try to adjust them in the dashboard
t_initial=19
t_intro_measures=33
t_hold=140
t_relax=4
beta_max=0.4
beta_min=0.11

# initial infected size
# coupling condition I0+S0+R0=N0
t=np.arange(len(ydata))
I0=ydata[0] 
S0=N0-I0
R0=0
record_start_index = 0

def SIR_model(SIR,beta,gamma):
    #print('SIR')
    ''' Simple SIR model
        S: susceptible population
        I: infected people
        R: recovered people
        beta: infection rate
        overall condition is that the sum of changes (differences) sum up to 0
        dS+dI+dR=0
        S+I+R= N 
    '''
    
    S,I,R=SIR
    dS_dt=-beta*S*I/N0     # (-) sign here because beta is getting smaller     
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    #print_test()
    return([dS_dt,dI_dt,dR_dt])

def SIR_model_t(SIR,t,beta,gamma): #beta,gamma are args
    #print('SIR_t')
    ''' 
        Function that returns derivative values at requested y and t values,
        it will be an input for the (integrate.odeint) function
        t: time step, mandatory for integral.odeint
    
    ''' 
    S,I,R=SIR
    dS_dt=-beta*S*I/N0           
    dI_dt=beta*S*I/N0-gamma*I
    dR_dt=gamma*I
    #print_test()
    return ([dS_dt,dI_dt,dR_dt])

def fit_odeint(x, beta, gamma):
    #print('fit_odient')
    '''
    helper function for the integration
    we only would like to get dI
    
    '''
    #print_test()
    return integrate.odeint(SIR_model_t, (S0, I0, R0), t, args=(beta, gamma))[:,1]


def static_fit_beta(S0,I0,R0,fit_odeint,t, ydata,beta,gamma):
    
    '''
    optimizing and fitting the parameters of the SIR model
    '''
    #print('Static_Beta_fit')
    SIR=np.array([S0,I0,R0]) # initialize sir statuse
    propagation_rates=pd.DataFrame(columns={'susceptible':S0, #initialize propagation rate
                                        'infected':I0,
                                        'recoverd':R0})

    for each_t in np.arange(100): # calculating the delta of sir model
   
            new_delta_vec=SIR_model(SIR,beta,gamma)
   
            SIR=SIR+new_delta_vec # update the current state with the new delta vec
    
            propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
    popt, pcov = optimize.curve_fit(fit_odeint, t, ydata) #we getback the fitted beta, gamma in the popt.vec(parameter optimized)
    perr = np.sqrt(np.diag(pcov))
       
    fitted=fit_odeint(t, *popt) # final fitted result
    propagation_rates= propagation_rates.drop(['recoverd'],axis=1)
    print('propagation_rates =', propagation_rates.head())
    print('optimal parameters =', popt)
    print('Fitted result =', fitted)
    #print_test()
   
    return fitted, popt
    
def dynamic_fit_beta(S0,I0,R0,t_initial,beta_max, beta_min,t_intro_measures,t_hold,t_relax):
    '''
    sampling individual parameters in dynamic simulation
    '''
    #print('Dynamic_Beta_fit')
        
    pd_beta=np.concatenate((np.array(t_initial*[beta_max]), # hard mesures t_intro_mesures
                       np.linspace(beta_max,beta_min,t_intro_measures),#linspace command to go from beta max to beta min
                       np.array(t_hold*[beta_min]),  
                        np.linspace(beta_min,beta_max,t_relax),
                       ))
        
    SIR=np.array([S0,I0,R0])
    propagation_rates=pd.DataFrame(columns={'susceptible':S0,
                                        'infected':I0,
                                        'recoverd':R0})

    for each_beta in pd_beta: # we go through the beta vector(dynamic beta)
            
                #print("SIR ",SIR, each_beta, gamma)
   
                new_delta_vec=SIR_model(SIR,each_beta,gamma)
            
                #print("New Delta Vec = ", new_delta_vec)
   
                SIR=SIR+new_delta_vec
    
                propagation_rates=propagation_rates.append({'susceptible':SIR[0],
                                                'infected':SIR[1],
                                                'recovered':SIR[2]}, ignore_index=True)
    
    
    x1=propagation_rates.index
    y1=propagation_rates.infected
        
    
    #print_test()
    print(pd_beta)
    return x1,y1

def print_test():
    print("in function")

static_fit_beta(S0,I0,R0,fit_odeint,t, ydata,beta,gamma)
dynamic_fit_beta(S0,I0,R0,t_initial,beta_max, beta_min,t_intro_measures,t_hold,t_relax)

C:\Users\Acer\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.



propagation_rates =      susceptible   infected  recovered
0  999977.600102  20.799898   1.600000
1  999969.280330  27.039681   3.679990
2  999958.464790  35.151252   6.383958
3  999944.404873  45.696044   9.899083
4  999926.127471  59.403841  14.468687
optimal parameters = [0.12208878 0.04213098]
Fitted result = [1.60000000e+01 1.73318257e+01 1.87745064e+01 2.03372685e+01
 2.20301060e+01 2.38638443e+01 2.58502097e+01 2.80019045e+01
 3.03326872e+01 3.28574615e+01 3.55923706e+01 3.85549010e+01
 4.17639936e+01 4.52401647e+01 4.90056372e+01 5.30844823e+01
 5.75027731e+01 6.22887506e+01 6.74730041e+01 7.30886662e+01
 7.91716239e+01 8.57607473e+01 9.28981371e+01 1.00629393e+02
 1.09003903e+02 1.18075159e+02 1.27901095e+02 1.38544458e+02
 1.50073205e+02 1.62560934e+02 1.76087352e+02 1.90738778e+02
 2.06608696e+02 2.23798339e+02 2.42417335e+02 2.62584396e+02
 2.84428073e+02 3.08087559e+02 3.33713571e+02 3.61469298e+02
 3.91531423e+02 4.24091234e+02 4.59355821e+02 4.97549367e+02
 5.38914555e+0

(RangeIndex(start=0, stop=196, step=1),
 0       20.799898
 1       27.039681
 2       35.151252
 3       45.696044
 4       59.403841
           ...    
 191    173.351292
 192    164.834835
 193    164.105488
 194    170.714848
 195    185.220303
 Name: infected, Length: 196, dtype: float64)

# 2. Evaluation & visualization
* Using the dash library to plot our data model with a user interactive dashboard

In [20]:

fig = go.Figure()
app = dash.Dash() # Launch the application

# Create a Dash layout that contains a Graph component
app.layout = html.Div([
    html.H3('SIR_Model_Dashboard'), # main title

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    This dashboard shows the confirmed infected COVID-19 cases with the static and dynamic fitting of the SIR _ model.

    '''),
    
    
    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),

    #first dropdown menu to selct the country
    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in df_input_large['country'].unique()],
        value='US', # which is pre-selected
        multi=False
    ),

    dcc.Markdown('''
        ## SIR_Model Static or Dynamic Beta fitting simulation
        '''),

    # second dropdown menu to select whether static fit or dynamic fit
    dcc.Dropdown(
    id='beta',
    options=[
        {'label': 'Static beta fit', 'value': 'static'},
        {'label': 'Dynamic beta fit', 'value': 'dynamic'},
    ],
    value='static',
    multi=False
    ),
    # the coming four input elements for the the four hard mesure parameters 
    dcc.Markdown('''
        ## Dynamic Simulation
        '''),
    html.Label('__t_initial__'), #1st input object
    dcc.Input(
    id = 't_initial_input',
    type='number',
    placeholder='t_initial',
    min=1, 
    max=150,
    step =1, 
    value= 19
    ), 
    
    html.Label('__t_intro_measures__'),#2nd input object
    dcc.Input(
    id = 't_intro_measures_input',
    type='number',
    placeholder='t_intro_measures',
    min=1, 
    max=150,
    step =1, 
    value= 33
    ), 
    
    html.Label('__t_hold__'),#3rd input object
    dcc.Input(
    id = 't_hold_input',
    type='number',
    placeholder='t_hold',
    min=1, 
    max=150,
    step =1, 
    value= 140
    ), 
    
    html.Label('__t_relax'),#4th input object
    dcc.Input(
    id = 't_relax_input',
    type='number',
    placeholder='t_relax',
    min=1, 
    max=150,
    step =1, 
    value= 4
    ),
    dcc.Markdown('''
        ## Beta Range
        '''),
    # range slider to adjust beta values 
    dcc.RangeSlider(
        id='beta_range_value',
        min=0.11,
        max=0.5,
        step=.01,
        value=[0.11, 0.29],
        marks={
        0.11: '0.11 Beta',
        0.16: '0.16 Beta',
        0.21: '0.21 Beta',
        0.26: '0.26 Beta',
        0.31: '0.31 Beta',
        0.36: '0.36 Beta',
        0.41: '0.41 Beta',
        0.46: '0.46 Beta',
        0.5: '0.5 Beta'
    },
    included=True,
    ),
    

    dcc.Graph(figure=fig, id='main_window_slope') # graph object
])

# app inputs & outputs are described in the arguments of the callback function
@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('beta', 'value'),
    Input('t_initial_input','value'),
    Input('t_intro_measures_input','value'),
    Input('t_hold_input','value'),
    Input('t_relax_input','value'),
    Input('beta_range_value','value')])

# update the figure traces with the new selected values from the dropdown menus, input items, and the range slider

def update_figure(country_list,show_beta,t_initial_input,t_intro_measures_input, t_hold_input, t_relax_input,beta_range_value):
    
    # reinitializing the country list selector
    df_plot=df_input_large[df_input_large['country']==country_list]
    ydata = np.array(df_plot.confirmed[25: ])
    
    header = "" # initialize the header
       
    my_yaxis={'type':"log",
                  'title':'Confirmed infected people (Model, log-scale)'
              }
    
    # updating traces according to user selection
    traces = []
    
    if show_beta=='static':
      
        fitted , popt = static_fit_beta(S0,I0,R0,fit_odeint,t,ydata,beta,gamma)   
       
        x1=t
        y1=fitted
        header= ' Optimal Beta = "{}" ,Gamma ="{}" , and Basic Reproduction Number R0 ="{}"'.format((popt[0]),(popt[1]),(popt[0]/ popt[1]))
        
        traces.append(dict(x=x1,
                                y=y1,
                                mode='markers+lines',
                                opacity=0.9,
                                name=country_list+" Model"
                        )
                )
        traces.append(dict(x=t,
                                y=ydata,
                                mode='markers+lines',
                                opacity=0.9,
                                name=country_list+" Confirmed"
                        )
                    )
    else:
        header= 'selected Beta values "{}"'.format(beta_range_value)
        
        x1,y1 = dynamic_fit_beta(S0,I0,R0,t_initial_input,beta_range_value[1], beta_range_value[0],t_intro_measures_input,t_hold_input,t_relax_input)
        
        traces.append(dict(x=x1,
                                y=y1,
                                mode='markers+lines',
                                opacity=0.9,
                                name=country_list+" Model"
                        )
                )
        traces.append(dict(x=t,
                                y=ydata,
                                mode='markers+lines',
                                opacity=0.9,
                                name=country_list+" Confirmed"
                        )
                    )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,
                title=header,
               
                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }
#to launch multiple dash apps, i had to change the port no from the first dah( here it is 8050)
app.run_server(debug=True,port=8050, use_reloader=False)  

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
Debugger PIN: 179-137-244
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


C:\Users\Acer\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:63: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\ipykernel_launcher.py:65: RuntimeWarning:

overflow encountered in double_scalars

C:\Users\Acer\anaconda3\lib\site-packages\scipy\integrate\odepack.py:247: ODEintWarning:

Illegal input detected (internal error). Run with full_output = 1 to get quantitative information.



propagation_rates =      susceptible   infected  recovered
0  999977.600102  20.799898   1.600000
1  999969.280330  27.039681   3.679990
2  999958.464790  35.151252   6.383958
3  999944.404873  45.696044   9.899083
4  999926.127471  59.403841  14.468687
optimal parameters = [ 0.17466855 -0.01341258]
Fitted result = [1.60000000e+01 1.93108437e+01 2.33067793e+01 2.81295619e+01
 3.39502754e+01 4.09753980e+01 4.94541242e+01 5.96871981e+01
 7.20375741e+01 8.69432768e+01 1.04932924e+02 1.26644447e+02
 1.52847678e+02 1.84471598e+02 2.22637196e+02 2.68697099e+02
 3.24283359e+02 3.91365051e+02 4.72317698e+02 5.70006914e+02
 6.87889146e+02 8.30132955e+02 1.00176494e+03 1.20884524e+03
 1.45867834e+03 1.76006621e+03 2.12361178e+03 2.56208234e+03
 3.09084391e+03 3.72837949e+03 4.49690562e+03 5.42310397e+03
 6.53898573e+03 7.88290855e+03 9.50076576e+03 1.14473671e+04
 1.37880271e+04 1.66003712e+04 1.99763585e+04 2.40245015e+04
 2.88722361e+04 3.46683533e+04 4.15853492e+04 4.98214673e+04
 5.96021126e

[0.29     0.29     0.29     0.29     0.29     0.29     0.29     0.29
 0.29     0.29     0.29     0.29     0.29     0.29     0.29     0.29
 0.29     0.29     0.29     0.284375 0.27875  0.273125 0.2675   0.261875
 0.25625  0.250625 0.245    0.239375 0.23375  0.228125 0.2225   0.216875
 0.21125  0.205625 0.2      0.194375 0.18875  0.183125 0.1775   0.171875
 0.16625  0.160625 0.155    0.149375 0.14375  0.138125 0.1325   0.126875
 0.12125  0.115625 0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11     0.11     0.11     0.11     0.11     0.11     0.11
 0.11     0.11    

[0.3        0.3        0.3        0.3        0.3        0.3
 0.3        0.3        0.3        0.3        0.3        0.3
 0.3        0.3        0.3        0.3        0.3        0.3
 0.3        0.29419355 0.2883871  0.28258065 0.27677419 0.27096774
 0.26516129 0.25935484 0.25354839 0.24774194 0.24193548 0.23612903
 0.23032258 0.22451613 0.21870968 0.21290323 0.20709677 0.20129032
 0.19548387 0.18967742 0.18387097 0.17806452 0.17225806 0.16645161
 0.16064516 0.15483871 0.14903226 0.14322581 0.13741935 0.1316129
 0.12580645 0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.12
 0.12       0.12       0.12       0.12       0.12       0.1